<a href="https://colab.research.google.com/github/samc2810/tools-and-links/blob/master/BigQuery_bquxjob_2bf1e3_18d64f14e7e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'hagel-shop-germany' # Project ID inserted based on the query results selected to explore
location = 'europe-west3' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [5]:
sql = """
select count(distinct email_hash) as cnt
from  `hagel-shop-germany.shopdata.cptmp_salesdata_test`
where extract(year FROM order_date)=2021;
"""

res = client.query(sql).to_dataframe()
print(res)

     cnt
0  16966


In [12]:
year2report = 2021

queries = {
    "customers_unique" : """
    select count(distinct email_hash)
from  `hagel-shop-germany.shopdata.cptmp_salesdata_test`
where extract(year FROM order_date)=year2report;
""",
    "purchase_value_total": """ """,
    "customers_new": """ """,
    "purchase_value_new": """ """,
    "customers_returning": """ """
}

queries = {
    "customers_unique" : """
    select count(distinct email_hash)
from  `shopdata.cptmp_salesdata_test`
where extract(year FROM order_date)="""+str(year2report),
    "purchase_value_total": """ select sum(row_total_incl_tax)
from  `shopdata.cptmp_salesdata_test`
where extract(year FROM order_date)= """+str(year2report),
}
results = {}
for key, q in queries.items():
  results[key] = None
  res_df = client.query(q).to_dataframe()
  val = res_df.iloc[0,0]
  results[key] = val


print(results)



{'customers_unique': 16966, 'purchase_value_total': 1145856.2599999704}


In [24]:
import json
import numpy as np
import datetime

def myconverter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()
results_parsed = {}
for col, val in results.items():
  results_parsed[col] = myconverter(val)

print(results_parsed)

{'customers_unique': 16966, 'purchase_value_total': 1145856.2599999704, 'year': None, 'is_active': None}


In [25]:
### Insert
dataset_insert = 'shopdata'
table_insert = 'aggregated_report'

results_parsed["year"] = year2report
results_parsed["is_active"] = True
insert_row_set = [results_parsed]
print(results_parsed, insert_row_set)
table_ref = client.get_table('{}.{}'.format(dataset_insert, table_insert))
print(table_ref)
#client.insert_rows(table_ref, [{"year": 2019, "is_active":False, "customers_unique": 2000}])
client.insert_rows(table_ref, insert_row_set)

{'customers_unique': 16966, 'purchase_value_total': 1145856.2599999704, 'year': 2021, 'is_active': True} [{'customers_unique': 16966, 'purchase_value_total': 1145856.2599999704, 'year': 2021, 'is_active': True}]
hagel-shop-germany.shopdata.aggregated_report


[]

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=hagel-shop-germany:europe-west3:bquxjob_2bf1e3_18d64f14e7e)
back to BigQuery to edit the query within the BigQuery user interface.

In [ ]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_2bf1e3_18d64f14e7e') # Job ID inserted based on the query results selected to explore
print(job.query)

# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [ ]:
# Running this code will read results from your previous job

job = client.get_job('bquxjob_2bf1e3_18d64f14e7e') # Job ID inserted based on the query results selected to explore
results = job.to_dataframe()
results

## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [ ]:
results.describe()